### Training code for dataset Bla Bla

Training is done in a supervised way. For every input patch, we have the two corresponding target patches using which we train our VSE (Variational Splitting Encoder decoder Network) with KL-divergence loss and a per-channel likelihood loss, following denoiSplit[ref]. In the likelihood computation, Noise models are used. Besides the primary input patch, we also feed LC inputs, originally introduced in uSplit[ref], to the network to make available information about larger spatial context. 

### Important ! 

This step can be skipped! Only run this notebook if you want to train the microsplit model from scratch or finetune. Pretrained model checkpoint is available 
By default this notebook runs example training for 5 epochs. 

#### General imports

In [ ]:
import pooch
from pathlib import Path
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from careamics.lightning import VAEModule

from microsplit_reproducibility.configs.factory import (
    create_algorithm_config,
    get_likelihood_config,
    get_loss_config,
    get_model_config,
    get_optimizer_config,
    get_training_config,
    get_lr_scheduler_config,
)
from microsplit_reproducibility.utils.callbacks import get_callbacks
from microsplit_reproducibility.utils.io import load_checkpoint, load_checkpoint_path
from microsplit_reproducibility.datasets import create_train_val_datasets
from microsplit_reproducibility.utils.utils import (
    plot_training_metrics,
    plot_input_patches,
    plot_training_outputs,
)

#### Experiments specific imports

In [ ]:
from microsplit_reproducibility.configs.parameters.HT_LIF24 import (
    get_microsplit_parameters,
)
from microsplit_reproducibility.configs.data.HT_LIF24 import get_data_configs
from microsplit_reproducibility.datasets.HT_LIF24 import get_train_val_data

### Download the data

In [ ]:
DATA = pooch.create(
    path="./data",
    base_url="https://download.fht.org/jug/ht_lif24",
    registry={"ht_lif24.zip": None},
)

NOISE_MODELS = pooch.create(
    path="./noise_models",
    base_url="https://download.fht.org/jug/ht_lif24",
    registry={
        "nm_ht_lif24_ch1_20ms.npz": None,
        "nm_ht_lif24_ch2_20ms.npz": None,
        "nm_ht_lif24_ch3_20ms.npz": None,
    },
)

MODEL_CHECKPOINTS = pooch.create(
    path="./checkpoints",
    base_url="https://download.fht.org/jug/ht_lif24",
    registry={"best.ckpt": None, "last.ckpt": None},
)

In [ ]:
for i, f in enumerate(NOISE_MODELS.registry):
    NOISE_MODELS.fetch(f"nm_ht_lif24_ch{i+1}_20ms.npz")

DATA.fetch("ht_lif24.zip", processor=pooch.Unzip())

for f in MODEL_CHECKPOINTS.registry:
    MODEL_CHECKPOINTS.fetch(f"{f}")

### Get data and experiment parameters

In [ ]:
train_data_config, val_data_config, test_data_configs = get_data_configs(
    dset_type="20ms"
)
experiment_params = get_microsplit_parameters(
    dset_type="20ms", nm_path=NOISE_MODELS.path
)

In [ ]:
for k in train_data_config:
    print(k)

In [ ]:
experiment_params

### Create dataset

In [ ]:
train_dset, val_dset, _, data_stats = create_train_val_datasets(
    datapath=DATA.path / "ht_lif24.zip.unzip/ht_lif24",
    train_config=train_data_config,
    val_config=val_data_config,
    test_config=val_data_config,
    load_data_func=get_train_val_data,
)

### Create dataloaders

In [ ]:
train_dloader = DataLoader(
    train_dset,
    batch_size=experiment_params["batch_size"],
    num_workers=experiment_params["num_workers"],
    shuffle=True,
)
val_dloader = DataLoader(
    val_dset,
    batch_size=experiment_params["batch_size"],
    num_workers=experiment_params["num_workers"],
    shuffle=False,
)

### Get experiment configs

In [ ]:
experiment_params["data_stats"] = data_stats  # TODO rethink

loss_config = get_loss_config(**experiment_params)
model_config = get_model_config(**experiment_params)
gaussian_lik_config, noise_model_config, nm_lik_config = get_likelihood_config(
    **experiment_params
)
training_config = get_training_config(**experiment_params)
lr_scheduler_config = get_lr_scheduler_config(**experiment_params)
optimizer_config = get_optimizer_config(**experiment_params)

experiment_config = create_algorithm_config(
    algorithm=experiment_params["algorithm"],
    loss_config=loss_config,
    model_config=model_config,
    gaussian_lik_config=gaussian_lik_config,
    nm_config=noise_model_config,
    nm_lik_config=nm_lik_config,
    lr_scheduler_config=lr_scheduler_config,
    optimizer_config=optimizer_config,
)

### Initialize the model

In [ ]:
model = VAEModule(algorithm_config=experiment_config)

### Load checkpoint (optional)

It's possible to load a checkpoint to continue training

In [ ]:
ckpt_path = load_checkpoint_path("checkpoints", best=True)
model = VAEModule.load_from_checkpoint(
    ckpt_path, algorithm_config=experiment_config
)

### Visualize input data


In [ ]:
plot_input_patches(dataset=train_dset, num_channels=3, num_samples=3)

### Train the model

Only 5 epochs for the sake of the example

In [ ]:
trainer = Trainer(
    max_epochs=training_config.num_epochs,
    accelerator="gpu",
    enable_progress_bar=True,
    callbacks=get_callbacks("/checkpoints"),
    precision=training_config.precision,
    gradient_clip_val=training_config.gradient_clip_val,
    gradient_clip_algorithm=training_config.gradient_clip_algorithm,
)
trainer.fit(
    model=model,
    train_dataloaders=train_dloader,
    val_dataloaders=val_dloader,
)

### Training logs

In [ ]:
plot_training_metrics(list(Path("csv_logs").rglob("metrics.csv"))[0])

In [ ]:
plot_training_outputs(val_dset, trainer.model, num_channels=3)